# Part1 : Data Processing

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')

In [6]:
#Identify inputs and output
X = dataset.iloc[:,3:13].values
y = dataset.iloc[:, 13].values

In [8]:
#Categorical variables into numerical values
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


#Encode the country name
#Instantiate
labelencoder_X_1 = LabelEncoder()
#Convert
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])

In [9]:
#Encode gender
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

In [12]:
#Create dummy variables for country variables as it contains 3 categories
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()

In [13]:
#Remove one of the dummy variable as it will be implicit
X = X[:,1:]

In [15]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [16]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# PART 2 : ANN

In [19]:
import keras
#To initialize ANN
from keras.models import Sequential
#To build layers in ANN
from keras.layers import Dense
#To prevent over-fitting, we use dropout - At each iteration, some of the neurons r randomly disabled to prevent them from 
#Being too dependent on each other
#U can choose which layer to apply
from Keras.layers import Dropout

In [20]:
#Initializing the ANN
classifier = Sequential()

In [21]:
#Add the input layer and first hidden layer
#Rule of thumb : Number of hidden layer = (no of input layer + output layer )/2
classifier.add(Dense(input_dim=11,output_dim = 6, init = 'uniform', activation='relu'))
#Try different values of p and check for overfitting, shoulb be between 1 and 4
classifier.add(Dropout(p=0.1))

C:\Users\amuly\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=6, kernel_initializer="uniform", input_dim=11)`
  app.launch_new_instance()


In [22]:
#Add the second hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation='relu'))
classifier.add(Dropout(p=0.1))

C:\Users\amuly\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=6, kernel_initializer="uniform")`
  from ipykernel import kernelapp as app


In [23]:
#Add the output layer
classifier.add(Dense(output_dim = 1, init = 'uniform', activation='sigmoid'))

C:\Users\amuly\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  from ipykernel import kernelapp as app


In [24]:
#Compiling the ANN
#Read about metrics
classifier.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])

In [27]:
#Fitting model to training set
#Batch size for batch stocastic gradient decent
classifier.fit(X_train,y_train,batch_size=10,nb_epoch=10)
#The accuracy value should converge after some epochs

Epoch 1/10
 580/8000 [=>............................] - ETA: 2s - loss: 0.4263 - acc: 0.8086

C:\Users\amuly\Anaconda3\lib\site-packages\keras\models.py:844: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


8000/8000 [==============================] - 1s - loss: 0.4017 - acc: 0.8346     
Epoch 2/10
8000/8000 [==============================] - 1s - loss: 0.4020 - acc: 0.8349     
Epoch 3/10
8000/8000 [==============================] - 1s - loss: 0.4020 - acc: 0.8349     
Epoch 4/10
8000/8000 [==============================] - 1s - loss: 0.4016 - acc: 0.8342     
Epoch 5/10
8000/8000 [==============================] - 2s - loss: 0.4016 - acc: 0.8355     
Epoch 6/10
8000/8000 [==============================] - 1s - loss: 0.4016 - acc: 0.8337     
Epoch 7/10
8000/8000 [==============================] - 1s - loss: 0.4015 - acc: 0.8340     
Epoch 8/10
8000/8000 [==============================] - 1s - loss: 0.4013 - acc: 0.8339     
Epoch 9/10
8000/8000 [==============================] - 2s - loss: 0.4017 - acc: 0.8352     
Epoch 10/10
8000/8000 [==============================] - 1s - loss: 0.4015 - acc: 0.8354     


# Part 3 : Evaluate the model

In [29]:
#Predict the output, since its a sigmoid, we have to define a treshold for binary classification
y_pred = classifier.predict(X_test)

#To set the threshold
y_pred = (y_pred > 0.5)

In [37]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1548,   47],
       [ 270,  135]])

In [36]:
#Calculate the accuracy!
accuracy = (cm[0,0] + cm[1,1])/2000
accuracy

0.84150000000000003

In [41]:
#Predict a new output
#Use the same scale and encoding
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))

In [42]:
#Calculate the op
(new_prediction > 0.5)

array([[False]], dtype=bool)

# Part 4 : Evaluate and Improve the model

In [46]:
#Keras wrapper for sklearn
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [47]:
# KerasClassifier has an input which is a function
def build_classifier() :
    classifier = Sequential()
    classifier.add(Dense(input_dim=11,output_dim = 6, init = 'uniform', activation='relu'))
    classifier.add(Dense(output_dim = 6, init = 'uniform', activation='relu'))
    classifier.add(Dense(output_dim = 1, init = 'uniform', activation='sigmoid'))
    classifier.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])
    return classifier

In [48]:
#New Classifier trained with cross validation
classifier = KerasClassifier(build_fn=build_classifier,batch_size=10,nb_epoch=10)

In [ ]:
#Cross Validation
#For parallel computation -> n_jobs=-1
accuracies = cross_val_score(estimator=classifier,X=X_train,y=y_train,cv=5,n_jobs=-1)

In [ ]:
#CHeck for low bias and Variance
mean = accuracies.mean()
var = accuracies.std()

# Part 5 : Improve ANN / Parameter tuning

In [ ]:
#Added DROPOUT

In [ ]:
#Hyper-parameters :- no of epochs, batch size, optimizer, no of nuerons

In [1]:
#GridSearch for selecting parameters
from sklearn.model_selection import GridSearchCV
def build_classifier(optimizer) :
    classifier = Sequential()
    classifier.add(Dense(input_dim=11,output_dim = 6, init = 'uniform', activation='relu'))
    classifier.add(Dense(output_dim = 6, init = 'uniform', activation='relu'))
    classifier.add(Dense(output_dim = 1, init = 'uniform', activation='sigmoid'))
    classifier.compile(optimizer = optimizer,loss = 'binary_crossentropy',metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn=build_classifier)

In [ ]:
parameters = {'batch_size' : [25,32],
             'nb_epoch' : [100,500],
             'optimizer' : ['adam','rmsprop']}

In [ ]:
grid_search = GrisSearchCV(estimator = classifier, param_grid = parameters,
                          scoring = 'accuracy', cv = 10)

In [ ]:
grid_search = grid_search.fit(X_train,y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_